# 第5课：数据清洗实战

## 学习目标
- 掌握缺失值处理方法
- 学会处理重复数据
- 理解异常值检测与处理
- 掌握数据类型转换

## 1. 创建示例数据

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 创建带问题的数据集
np.random.seed(42)

data = {
    'id': range(1, 101),
    'name': ['User_' + str(i) for i in range(1, 101)],
    'age': np.random.randint(18, 70, 100).astype(float),
    'salary': np.random.randint(3000, 20000, 100).astype(float),
    'department': np.random.choice(['IT', 'HR', 'Sales', 'Marketing', None], 100),
    'join_date': pd.date_range('2020-01-01', periods=100, freq='D').astype(str),
    'email': ['user' + str(i) + '@company.com' for i in range(1, 101)]
}

df = pd.DataFrame(data)

# 人为添加问题数据
df.loc[5, 'age'] = np.nan
df.loc[15, 'age'] = np.nan
df.loc[25, 'salary'] = np.nan
df.loc[35, 'salary'] = np.nan
df.loc[45, 'age'] = -5  # 异常值
df.loc[55, 'age'] = 150  # 异常值
df.loc[65, 'salary'] = 500000  # 异常值
df.loc[10, 'email'] = 'invalid-email'  # 格式错误
df.loc[20, 'join_date'] = '2020/01/21'  # 日期格式不一致

# 添加重复行
df = pd.concat([df, df.iloc[[1, 2, 3]]], ignore_index=True)

print("原始数据形状:", df.shape)
print("\n数据预览:")
print(df.head(10))

## 2. 数据质量检查

In [ ]:
# 数据概览
print("=" * 50)
print("数据基本信息")
print("=" * 50)
print(df.info())

In [ ]:
# 缺失值检查
print("\n缺失值统计:")
missing = df.isnull().sum()
missing_pct = (df.isnull().sum() / len(df) * 100).round(2)
missing_df = pd.DataFrame({'缺失数量': missing, '缺失比例(%)': missing_pct})
print(missing_df[missing_df['缺失数量'] > 0])

In [ ]:
# 重复值检查
print(f"\n重复行数量: {df.duplicated().sum()}")
print("\n重复的行:")
print(df[df.duplicated(keep=False)].sort_values('id'))

In [ ]:
# 数值统计
print("\n数值型列统计:")
print(df.describe())

## 3. 缺失值处理

In [ ]:
# 复制数据用于处理
df_clean = df.copy()

# 方法1：删除缺失值
print("删除前行数:", len(df_clean))
df_dropped = df_clean.dropna()
print("删除缺失值后行数:", len(df_dropped))

In [ ]:
# 方法2：填充缺失值
df_clean = df.copy()

# 用均值填充数值列
df_clean['age'] = df_clean['age'].fillna(df_clean['age'].mean())
df_clean['salary'] = df_clean['salary'].fillna(df_clean['salary'].median())

# 用众数填充分类列
df_clean['department'] = df_clean['department'].fillna(df_clean['department'].mode()[0])

print("填充后缺失值:")
print(df_clean.isnull().sum())

In [ ]:
# 方法3：前向/后向填充
df_temp = pd.DataFrame({
    'date': pd.date_range('2024-01-01', periods=10),
    'value': [1, np.nan, np.nan, 4, 5, np.nan, 7, 8, np.nan, 10]
})

print("原始数据:")
print(df_temp)

print("\n前向填充 (ffill):")
print(df_temp.fillna(method='ffill'))

print("\n后向填充 (bfill):")
print(df_temp.fillna(method='bfill'))

In [ ]:
# 方法4：插值填充
print("线性插值:")
df_temp['interpolated'] = df_temp['value'].interpolate(method='linear')
print(df_temp)

## 4. 重复数据处理

In [ ]:
# 查看重复数据
print(f"处理前行数: {len(df_clean)}")
print(f"重复行数: {df_clean.duplicated().sum()}")

# 删除完全重复的行
df_clean = df_clean.drop_duplicates()
print(f"删除重复后行数: {len(df_clean)}")

In [ ]:
# 基于特定列去重
df_example = pd.DataFrame({
    'name': ['Alice', 'Bob', 'Alice', 'Charlie'],
    'age': [25, 30, 26, 35],
    'city': ['NY', 'LA', 'NY', 'Chicago']
})

print("原始数据:")
print(df_example)

print("\n按 name 去重 (保留第一个):")
print(df_example.drop_duplicates(subset=['name'], keep='first'))

print("\n按 name 去重 (保留最后一个):")
print(df_example.drop_duplicates(subset=['name'], keep='last'))

## 5. 异常值处理

In [ ]:
# 可视化异常值
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

sns.boxplot(data=df_clean, y='age', ax=axes[0])
axes[0].set_title('Age 箱线图')

sns.boxplot(data=df_clean, y='salary', ax=axes[1])
axes[1].set_title('Salary 箱线图')

plt.tight_layout()
plt.show()

In [ ]:
# 方法1：IQR 方法检测异常值
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower) | (data[column] > upper)]
    return outliers, lower, upper

# 检测 age 异常值
outliers_age, lower_age, upper_age = detect_outliers_iqr(df_clean, 'age')
print(f"Age 异常值范围: < {lower_age:.2f} 或 > {upper_age:.2f}")
print(f"Age 异常值数量: {len(outliers_age)}")
print(outliers_age[['id', 'name', 'age']])

In [ ]:
# 方法2：Z-Score 方法
from scipy import stats

def detect_outliers_zscore(data, column, threshold=3):
    z_scores = np.abs(stats.zscore(data[column].dropna()))
    outlier_indices = np.where(z_scores > threshold)[0]
    return data.iloc[outlier_indices]

outliers_salary = detect_outliers_zscore(df_clean, 'salary')
print(f"Salary Z-Score 异常值数量: {len(outliers_salary)}")
print(outliers_salary[['id', 'name', 'salary']])

In [ ]:
# 处理异常值
# 方法1：删除异常值
df_no_outliers = df_clean[
    (df_clean['age'] >= lower_age) & 
    (df_clean['age'] <= upper_age)
].copy()

print(f"删除异常值前: {len(df_clean)} 行")
print(f"删除异常值后: {len(df_no_outliers)} 行")

In [ ]:
# 方法2：截断/裁剪异常值
df_clipped = df_clean.copy()
df_clipped['age'] = df_clipped['age'].clip(lower=0, upper=100)
df_clipped['salary'] = df_clipped['salary'].clip(lower=0, upper=50000)

print("裁剪后的统计:")
print(df_clipped[['age', 'salary']].describe())

In [ ]:
# 方法3：用边界值替换
df_replaced = df_clean.copy()

# 用上下界替换异常值
df_replaced.loc[df_replaced['age'] < lower_age, 'age'] = lower_age
df_replaced.loc[df_replaced['age'] > upper_age, 'age'] = upper_age

print("替换后年龄范围:", df_replaced['age'].min(), "-", df_replaced['age'].max())

## 6. 数据类型转换

In [ ]:
df_final = df_clipped.copy()

print("转换前数据类型:")
print(df_final.dtypes)

In [ ]:
# 转换数值类型
df_final['age'] = df_final['age'].astype(int)
df_final['salary'] = df_final['salary'].astype(int)

# 转换日期类型
df_final['join_date'] = pd.to_datetime(df_final['join_date'], errors='coerce')

# 转换分类类型
df_final['department'] = df_final['department'].astype('category')

print("转换后数据类型:")
print(df_final.dtypes)

In [ ]:
# 提取日期信息
df_final['join_year'] = df_final['join_date'].dt.year
df_final['join_month'] = df_final['join_date'].dt.month
df_final['join_day'] = df_final['join_date'].dt.day
df_final['join_weekday'] = df_final['join_date'].dt.day_name()

print("添加日期特征后:")
print(df_final[['join_date', 'join_year', 'join_month', 'join_day', 'join_weekday']].head())

## 7. 字符串清洗

In [ ]:
# 创建带问题的字符串数据
df_str = pd.DataFrame({
    'name': ['  Alice  ', 'BOB', 'charlie', '  David Lee  '],
    'phone': ['123-456-7890', '(234) 567-8901', '345.678.9012', '456 789 0123'],
    'email': ['alice@email.com', 'bob@EMAIL.COM', 'invalid', 'david@email.com']
})

print("原始数据:")
print(df_str)

In [ ]:
# 字符串清洗
# 去除空格
df_str['name_clean'] = df_str['name'].str.strip()

# 统一大小写
df_str['name_clean'] = df_str['name_clean'].str.title()

# 统一电话格式
df_str['phone_clean'] = df_str['phone'].str.replace(r'[^0-9]', '', regex=True)

# 统一 email 小写
df_str['email_clean'] = df_str['email'].str.lower()

print("清洗后:")
print(df_str)

In [ ]:
# 验证 email 格式
import re

def is_valid_email(email):
    pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    return bool(re.match(pattern, str(email)))

df_str['email_valid'] = df_str['email_clean'].apply(is_valid_email)
print("Email 验证结果:")
print(df_str[['email_clean', 'email_valid']])

## 8. 数据标准化与归一化

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# 准备数值数据
numeric_data = df_final[['age', 'salary']].copy()

# 标准化 (Z-Score)
scaler_standard = StandardScaler()
numeric_data[['age_standardized', 'salary_standardized']] = scaler_standard.fit_transform(
    numeric_data[['age', 'salary']]
)

# 归一化 (Min-Max)
scaler_minmax = MinMaxScaler()
numeric_data[['age_normalized', 'salary_normalized']] = scaler_minmax.fit_transform(
    numeric_data[['age', 'salary']]
)

print("标准化和归一化结果:")
print(numeric_data.head(10))

In [ ]:
# 可视化对比
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].hist(numeric_data['salary'], bins=20)
axes[0].set_title('原始 Salary')

axes[1].hist(numeric_data['salary_standardized'], bins=20)
axes[1].set_title('标准化 Salary')

axes[2].hist(numeric_data['salary_normalized'], bins=20)
axes[2].set_title('归一化 Salary')

plt.tight_layout()
plt.show()

## 9. 数据清洗流水线

In [ ]:
def clean_data(df):
    """数据清洗流水线"""
    df_clean = df.copy()
    
    # 1. 删除重复行
    df_clean = df_clean.drop_duplicates()
    print(f"1. 删除重复后: {len(df_clean)} 行")
    
    # 2. 处理缺失值
    # 数值列用中位数填充
    numeric_cols = df_clean.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        df_clean[col] = df_clean[col].fillna(df_clean[col].median())
    
    # 分类列用众数填充
    cat_cols = df_clean.select_dtypes(include=['object', 'category']).columns
    for col in cat_cols:
        if df_clean[col].isnull().any():
            df_clean[col] = df_clean[col].fillna(df_clean[col].mode()[0])
    print(f"2. 填充缺失值完成")
    
    # 3. 处理异常值（裁剪到合理范围）
    if 'age' in df_clean.columns:
        df_clean['age'] = df_clean['age'].clip(0, 100)
    if 'salary' in df_clean.columns:
        df_clean['salary'] = df_clean['salary'].clip(0, 100000)
    print(f"3. 异常值处理完成")
    
    # 4. 转换数据类型
    if 'age' in df_clean.columns:
        df_clean['age'] = df_clean['age'].astype(int)
    if 'salary' in df_clean.columns:
        df_clean['salary'] = df_clean['salary'].astype(int)
    print(f"4. 数据类型转换完成")
    
    return df_clean

# 应用清洗流水线
df_processed = clean_data(df)
print(f"\n最终数据形状: {df_processed.shape}")
print("\n数据质量检查:")
print(f"缺失值: {df_processed.isnull().sum().sum()}")
print(f"重复行: {df_processed.duplicated().sum()}")

## 10. 练习题

### 练习：清洗以下数据集

In [ ]:
# 创建练习数据
messy_data = pd.DataFrame({
    'product': ['Apple', 'APPLE', '  banana  ', 'Orange', 'apple', None],
    'price': [1.5, 1.5, 0.8, np.nan, 1.6, 2.0],
    'quantity': [100, 100, 50, 30, -10, 200],
    'date': ['2024-01-01', '2024/01/02', '01-03-2024', '2024-01-04', '2024-01-05', '2024-01-06']
})

print("原始数据:")
print(messy_data)

# 在这里编写清洗代码


## 11. 本课小结

1. **缺失值处理**：删除、填充（均值/中位数/众数）、插值
2. **重复数据**：drop_duplicates
3. **异常值**：IQR、Z-Score 检测，裁剪/删除/替换
4. **类型转换**：astype、pd.to_datetime
5. **字符串清洗**：strip、lower/upper、正则替换
6. **标准化/归一化**：StandardScaler、MinMaxScaler